In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [5]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [6]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.28134798
====> Test set loss: 1.2909, 62.0%
====> Epoch: 150 Average loss: 1.20173033
====> Test set loss: 1.2127, 69.0%
====> Epoch: 225 Average loss: 1.20760253
====> Test set loss: 1.2092, 69.0%
====> Epoch: 300 Average loss: 1.21259918
====> Test set loss: 1.2043, 69.0%
====> Epoch: 375 Average loss: 1.20176897
====> Test set loss: 1.2014, 70.0%
====> Epoch: 450 Average loss: 1.24033634
====> Test set loss: 1.2018, 69.5%
====> Epoch: 525 Average loss: 1.24107657
====> Test set loss: 1.1998, 70.5%
====> Epoch: 600 Average loss: 1.23011834
====> Test set loss: 1.1991, 70.5%
====> Epoch: 675 Average loss: 1.22299410
====> Test set loss: 1.1984, 70.5%
====> Epoch: 750 Average loss: 1.21179613
====> Test set loss: 1.1980, 70.5%
Training state:  False
Elapsed:  46.78293299674988
Complete set accuracy: 72.0%


### Runner

In [9]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(424, 450):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 400
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.26119464
====> Test set loss: 1.2289, 66.5%
====> Epoch: 150 Average loss: 1.19980763
====> Test set loss: 1.2306, 68.5%
====> Epoch: 225 Average loss: 1.18607877
====> Test set loss: 1.2280, 68.5%
====> Epoch: 300 Average loss: 1.13992366
====> Test set loss: 1.2248, 68.0%
====> Epoch: 375 Average loss: 1.14335087
====> Test set loss: 1.2227, 67.5%
====> Epoch: 450 Average loss: 1.13914466
====> Test set loss: 1.2228, 67.5%
====> Epoch: 525 Average loss: 1.16966783
====> Test set loss: 1.2233, 67.5%
====> Epoch: 600 Average loss: 1.17463286
====> Test set loss: 1.2229, 67.5%
====> Epoch: 675 Average loss: 1.14481936
====> Test set loss: 1.2236, 67.5%
====> Epoch: 750 Average loss: 1.14435693
====> Test set loss: 1.2233, 67.5%
Training state:  False
Complete set accuracy: 74.3%
Log accuracy: 72.3%
---- Done in  53.03826689720154  seconds

---- Running for model name:  B_add_mild_nlin
==

====> Epoch: 750 Average loss: 1.24108171
====> Test set loss: 1.1704, 71.5%
Training state:  False
Complete set accuracy: 72.39999999999999%
Log accuracy: 67.7%
---- Done in  56.92047309875488  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.30499724
====> Test set loss: 1.2754, 64.5%
====> Epoch: 150 Average loss: 1.32856666
====> Test set loss: 1.2139, 66.5%
====> Epoch: 225 Average loss: 1.27293593
====> Test set loss: 1.1997, 68.5%
====> Epoch: 300 Average loss: 1.24248090
====> Test set loss: 1.1956, 68.0%
====> Epoch: 375 Average loss: 1.25554879
====> Test set loss: 1.2007, 69.0%
====> Epoch: 450 Average loss: 1.23171150
====> Test set loss: 1.1988, 68.5%
====> Epoch: 525 Average loss: 1.24967293
====> Test set loss: 1.1976, 68.0%
====> Epoch: 600 Average loss: 1.27943318
====> Test set loss: 1.1969, 67.5%
====> Epoch: 675 Average loss: 1.28121738
====> Test set loss: 1.1961, 67.5%
====> Epoch: 750 Average loss: 1.24472427
====> Test set lo

====> Epoch: 525 Average loss: 1.08898021
====> Test set loss: 1.0532, 74.5%
====> Epoch: 600 Average loss: 1.08224910
====> Test set loss: 1.0525, 74.5%
====> Epoch: 675 Average loss: 1.13098666
====> Test set loss: 1.0525, 74.5%
====> Epoch: 750 Average loss: 1.10055753
====> Test set loss: 1.0516, 74.5%
Training state:  False
Complete set accuracy: 77.60000000000001%
Log accuracy: 75.5%
---- Done in  55.699743032455444  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.25716661
====> Test set loss: 1.2380, 66.0%
====> Epoch: 150 Average loss: 1.24433033
====> Test set loss: 1.1829, 67.5%
====> Epoch: 225 Average loss: 1.19045975
====> Test set loss: 1.1772, 68.0%
====> Epoch: 300 Average loss: 1.17627840
====> Test set loss: 1.1790, 68.0%
====> Epoch: 375 Average loss: 1.18225005
====> Test set loss: 1.1722, 68.0%
====> Epoch: 450 Average loss: 1.20334157
====> Test set loss: 1.1715, 68.0%
====> Epoch: 525 Average loss: 1.20976552
====> Tes

====> Epoch: 300 Average loss: 1.16750048
====> Test set loss: 1.1953, 72.0%
====> Epoch: 375 Average loss: 1.16455855
====> Test set loss: 1.1842, 72.5%
====> Epoch: 450 Average loss: 1.20173945
====> Test set loss: 1.1844, 72.5%
====> Epoch: 525 Average loss: 1.20556296
====> Test set loss: 1.1849, 72.5%
====> Epoch: 600 Average loss: 1.22070729
====> Test set loss: 1.1852, 72.5%
====> Epoch: 675 Average loss: 1.19460561
====> Test set loss: 1.1850, 72.5%
====> Epoch: 750 Average loss: 1.19631848
====> Test set loss: 1.1860, 72.5%
Training state:  False
Complete set accuracy: 73.4%
Log accuracy: 72.2%
---- Done in  58.76948404312134  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.25437438
====> Test set loss: 1.1977, 66.5%
====> Epoch: 150 Average loss: 1.24164844
====> Test set loss: 1.2024, 64.0%
====> Epoch: 225 Average loss: 1.16055027
====> Test set loss: 1.1739, 69.0%
====> Epoch: 300 Average loss: 1.17581234
====> Test set loss: 1.16

====> Epoch: 75 Average loss: 1.26610782
====> Test set loss: 1.1637, 75.5%
====> Epoch: 150 Average loss: 1.18053430
====> Test set loss: 1.1169, 75.5%
====> Epoch: 225 Average loss: 1.22351755
====> Test set loss: 1.1152, 73.5%
====> Epoch: 300 Average loss: 1.16643088
====> Test set loss: 1.1162, 73.0%
====> Epoch: 375 Average loss: 1.17078162
====> Test set loss: 1.1179, 73.0%
====> Epoch: 450 Average loss: 1.17199299
====> Test set loss: 1.1179, 73.0%
====> Epoch: 525 Average loss: 1.19039142
====> Test set loss: 1.1174, 73.0%
====> Epoch: 600 Average loss: 1.16370106
====> Test set loss: 1.1177, 73.0%
====> Epoch: 675 Average loss: 1.15170467
====> Test set loss: 1.1170, 73.0%
====> Epoch: 750 Average loss: 1.19747187
====> Test set loss: 1.1169, 73.0%
Training state:  False
Complete set accuracy: 73.5%
Log accuracy: 71.2%
---- Done in  57.96884107589722  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.21689977
====> Test set loss: 1.2138, 7

...

====> Epoch: 75 Average loss: 1.23230720
====> Test set loss: 1.2221, 68.0%
====> Epoch: 150 Average loss: 1.17561492
====> Test set loss: 1.1799, 69.0%
====> Epoch: 225 Average loss: 1.20323374
====> Test set loss: 1.1822, 70.0%
====> Epoch: 300 Average loss: 1.17583149
====> Test set loss: 1.1811, 70.5%
====> Epoch: 375 Average loss: 1.21529352
====> Test set loss: 1.1775, 71.0%
====> Epoch: 450 Average loss: 1.15792336
====> Test set loss: 1.1775, 71.0%
====> Epoch: 525 Average loss: 1.14839402
====> Test set loss: 1.1774, 71.0%
====> Epoch: 600 Average loss: 1.18299015
====> Test set loss: 1.1775, 71.0%
====> Epoch: 675 Average loss: 1.19177368
====> Test set loss: 1.1775, 71.0%
====> Epoch: 750 Average loss: 1.13483732
====> Test set loss: 1.1775, 71.0%
Training state:  False
Complete set accuracy: 75.2%
Log accuracy: 73.6%
---- Done in  61.15661811828613  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.24510838
====> Test set loss: 1.1128, 7

====> Epoch: 750 Average loss: 1.21136355
====> Test set loss: 1.2630, 63.0%
Training state:  False
Complete set accuracy: 73.1%
Log accuracy: 69.8%
---- Done in  68.44426417350769  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.24903018
====> Test set loss: 1.1730, 70.0%
====> Epoch: 150 Average loss: 1.17721731
====> Test set loss: 1.0905, 72.0%
====> Epoch: 225 Average loss: 1.16807020
====> Test set loss: 1.0864, 72.0%
====> Epoch: 300 Average loss: 1.17891724
====> Test set loss: 1.0864, 72.0%
====> Epoch: 375 Average loss: 1.16086863
====> Test set loss: 1.0768, 72.5%
====> Epoch: 450 Average loss: 1.18734632
====> Test set loss: 1.0780, 72.5%
====> Epoch: 525 Average loss: 1.17452956
====> Test set loss: 1.0789, 72.5%
====> Epoch: 600 Average loss: 1.20683070
====> Test set loss: 1.0793, 72.5%
====> Epoch: 675 Average loss: 1.20357281
====> Test set loss: 1.0794, 72.5%
====> Epoch: 750 Average loss: 1.19000807
====> Test set loss: 1.0797, 7

====> Epoch: 525 Average loss: 1.08459774
====> Test set loss: 1.0109, 76.0%
====> Epoch: 600 Average loss: 1.08887774
====> Test set loss: 1.0111, 76.0%
====> Epoch: 675 Average loss: 1.12410859
====> Test set loss: 1.0112, 76.0%
====> Epoch: 750 Average loss: 1.11734118
====> Test set loss: 1.0111, 76.0%
Training state:  False
Complete set accuracy: 79.5%
Log accuracy: 77.10000000000001%
---- Done in  70.964102268219  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.24681158
====> Test set loss: 1.1290, 74.5%
====> Epoch: 150 Average loss: 1.17483780
====> Test set loss: 1.0679, 73.0%
====> Epoch: 225 Average loss: 1.17671124
====> Test set loss: 1.0610, 73.5%
====> Epoch: 300 Average loss: 1.13764156
====> Test set loss: 1.0549, 74.5%
====> Epoch: 375 Average loss: 1.16380051
====> Test set loss: 1.0556, 74.5%
====> Epoch: 450 Average loss: 1.17240843
====> Test set loss: 1.0554, 74.5%
====> Epoch: 525 Average loss: 1.15841015
====> Test s

====> Epoch: 300 Average loss: 1.12326005
====> Test set loss: 1.1850, 70.0%
====> Epoch: 375 Average loss: 1.09966803
====> Test set loss: 1.1881, 70.0%
====> Epoch: 450 Average loss: 1.12653748
====> Test set loss: 1.1889, 70.0%
====> Epoch: 525 Average loss: 1.13040096
====> Test set loss: 1.1883, 70.0%
====> Epoch: 600 Average loss: 1.12779304
====> Test set loss: 1.1886, 70.0%
====> Epoch: 675 Average loss: 1.11801050
====> Test set loss: 1.1878, 70.0%
====> Epoch: 750 Average loss: 1.15971534
====> Test set loss: 1.1880, 70.0%
Training state:  False
Complete set accuracy: 77.8%
Log accuracy: 72.6%
---- Done in  68.32440090179443  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.24296338
====> Test set loss: 1.2057, 70.5%
====> Epoch: 150 Average loss: 1.21380679
====> Test set loss: 1.1789, 69.0%
====> Epoch: 225 Average loss: 1.23795026
====> Test set loss: 1.1754, 70.0%
====> Epoch: 300 Average loss: 1.24486903
====> Test set loss: 1.17

KeyboardInterrupt: 